In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from transformers import ViTModel, ViTFeatureExtractor

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        features = model(**inputs).last_hidden_state[:, 0, :]

    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 11399/11399 [22:59<00:00,  8.27it/s]


Extracted feature shape: (11399, 768)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(64,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 67.54% | Prec: 72.55% | Rec: 67.54% | F1: 68.25% | Bal Acc: 69.86% | MCC: 0.6444
LogisticRegression        | Acc: 94.08% | Prec: 93.64% | Rec: 94.08% | F1: 93.72% | Bal Acc: 78.36% | MCC: 0.9332
SVMLinear                 | Acc: 96.05% | Prec: 95.97% | Rec: 96.05% | F1: 95.90% | Bal Acc: 84.75% | MCC: 0.9555
SVMPolynomial             | Acc: 92.98% | Prec: 92.54% | Rec: 92.98% | F1: 92.42% | Bal Acc: 73.06% | MCC: 0.9208
SVMRadialBasis            | Acc: 91.93% | Prec: 91.29% | Rec: 91.93% | F1: 91.25% | Bal Acc: 68.82% | MCC: 0.9088
NearestNeighbor           | Acc: 84.43% | Prec: 84.41% | Rec: 84.43% | F1: 83.76% | Bal Acc: 63.98% | MCC: 0.8241
RandomForest              | Acc: 80.48% | Prec: 80.69% | Rec: 80.48% | F1: 78.50% | Bal Acc: 45.45% | MCC: 0.7791
MLP_Baseline              | Acc: 95.92% | Prec: 95.88% | Rec: 95.92% | F1: 95.78% | Bal Acc: 85.18% | MCC: 0.9540


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Calathus,Synuchus,Synuchus,Calathus,Calathus,Calathus,Calathus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Synuchus,Synuchus,Agonum,Synuchus,Synuchus,Synuchus,Calathus,Agonum


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/18.ViT-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/18.ViT-linear-probing-genus-metrics.csv", index=False)